<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 4 Dengue cases in Singapore by eweeks modelling/ prediction

## Problem Statement

## Importing Libraries


In [379]:
import json

In [2]:
# Imports:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

In [141]:
import pip
print(pip.__version__)
#$ sudo pip install --upgrade setuptools 
#pip install --upgrade pip
#python3 -m pip install --upgrade pip

9.0.1


In [142]:
#Attempted to update pip version and install pycaret but failed
#!pip install pycaret[full]


## Data Importing and Cleaning

#### Import Dengue data from MOH website
MOH data gives the full number of cases of infectious diseases in the whole Singapore, which includes Dengue Fever and Dengue Haemorhagic Fever (DHF).

In [380]:
filepath = 'data/'

In [381]:
df_daily_dengue = pd.read_csv(filepath + 'moh_data.csv')

In [382]:
df_daily_dengue.head()

,id,epi_week,epi_week_year,start,end,start_dd,start_mm,start_yyyy,end_dd,end_mm,...,zika,ebola,japanese_encephalitis,tetanus,botulism,murine_typhus,acute_upper_respiratory_tract_infections,acute_conjunctivitis,acute_diarrhoea,chickenpox
0,1,1,2012-W01,1/1/12,7/1/12,1,1,2012,7,1,...,0,0,0,0,0,0,2932,120,491,18
1,2,2,2012-W02,8/1/12,14/1/12,8,1,2012,14,1,...,0,0,0,0,0,0,3189,114,575,19
2,3,3,2012-W03,15/1/12,21/1/12,15,1,2012,21,1,...,0,0,0,0,0,0,3185,106,539,16
3,4,4,2012-W04,22/1/12,28/1/12,22,1,2012,28,1,...,0,0,0,0,0,0,4001,118,615,24
4,5,5,2012-W05,29/1/12,4/2/12,29,1,2012,4,2,...,0,0,0,0,0,0,3356,110,559,18


In [383]:
df_daily_dengue.shape

(570, 54)

#### Import Weather data from Singapore Meteorological Services
This was scraped from the web using selenium and gives daily weather data from 2012 to 2020 from 63 weather stations across the country.



In [384]:
df_weather = pd.read_csv(filepath + 'weather_cleaned.csv')

C:\Users\gohch\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [385]:
df_weather.shape

(228709, 15)

In [386]:
df_weather.head()

,Station,Year,Month,Day,Daily Rainfall Total (mm),Highest 30 Min Rainfall (mm),Highest 60 Min Rainfall (mm),Highest 120 Min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h),Date,Epiweek
0,Punggol,2012,1,1,9.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1/1/2012,2012-W01
1,Punggol,2012,1,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2/1/2012,2012-W01
2,Punggol,2012,1,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3/1/2012,2012-W01
3,Punggol,2012,1,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4/1/2012,2012-W01
4,Punggol,2012,1,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5/1/2012,2012-W01


In [387]:
df_weather.dtypes

Station                          object
Year                              int64
Month                             int64
Day                               int64
Daily Rainfall Total (mm)        object
Highest 30 Min Rainfall (mm)     object
Highest 60 Min Rainfall (mm)     object
Highest 120 Min Rainfall (mm)    object
Mean Temperature (°C)            object
Maximum Temperature (°C)         object
Minimum Temperature (°C)         object
Mean Wind Speed (km/h)           object
Max Wind Speed (km/h)            object
Date                             object
Epiweek                          object
dtype: object

In [388]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228709 entries, 0 to 228708
Data columns (total 15 columns):
Station                          228709 non-null object
Year                             228709 non-null int64
Month                            228709 non-null int64
Day                              228709 non-null int64
Daily Rainfall Total (mm)        223590 non-null object
Highest 30 Min Rainfall (mm)     169627 non-null object
Highest 60 Min Rainfall (mm)     169627 non-null object
Highest 120 Min Rainfall (mm)    169627 non-null object
Mean Temperature (°C)            106317 non-null object
Maximum Temperature (°C)         110711 non-null object
Minimum Temperature (°C)         110697 non-null object
Mean Wind Speed (km/h)           112110 non-null object
Max Wind Speed (km/h)            111638 non-null object
Date                             228709 non-null object
Epiweek                          228709 non-null object
dtypes: int64(3), object(12)
memory usage: 26.2+ MB


###### Dealing with missing values in Weather dataset
The simplest way would be to drop all the rows containing NA values, but that is not feasible as there are quite significant proportion of NA values in our data.
We observe there is approximately 30% missing data for columns related to Rainfall (Highest 30 Min, 60 Min and 120 Min Rainfall), and 60% missing data for columns related to Temperature (Mean, Max and Min Temperatures), and also 60% missing data for Wind speed columns (Mean Wind Speed (km/h), Max Wind Speed (km/h)).

###### Method for imputing NA values
- Rainfall: Take the mean of all the other values within the same year, month and Station. However, we note that there is no Rainfall data at all for 2012 - 2013, hence this is only possible for the year 2014 onwards.
- Temperature/ Wind Speed: Take the mean value of all other values within the same year, month and Station as well, so that the monthly averages are not affected. There may be higher number of NA's left after imputing as there are < 50% non-null values in these columns

First, we need to ensure that data types are float instead of string

In [389]:
columnlist_tofloat = ['Daily Rainfall Total (mm)', 'Highest 30 Min Rainfall (mm)', 'Highest 60 Min Rainfall (mm)', 
              'Highest 120 Min Rainfall (mm)', 'Mean Temperature (°C)', 'Maximum Temperature (°C)',
             'Minimum Temperature (°C)', 'Mean Wind Speed (km/h)', 'Max Wind Speed (km/h)']

In [390]:
for column in columnlist_tofloat:
    df_weather[column] = df_weather[column].apply(lambda x: float(x) if x!='-' else np.nan)

In [391]:
df_weather.dtypes

Station                           object
Year                               int64
Month                              int64
Day                                int64
Daily Rainfall Total (mm)        float64
Highest 30 Min Rainfall (mm)     float64
Highest 60 Min Rainfall (mm)     float64
Highest 120 Min Rainfall (mm)    float64
Mean Temperature (°C)            float64
Maximum Temperature (°C)         float64
Minimum Temperature (°C)         float64
Mean Wind Speed (km/h)           float64
Max Wind Speed (km/h)            float64
Date                              object
Epiweek                           object
dtype: object

In [392]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228709 entries, 0 to 228708
Data columns (total 15 columns):
Station                          228709 non-null object
Year                             228709 non-null int64
Month                            228709 non-null int64
Day                              228709 non-null int64
Daily Rainfall Total (mm)        220695 non-null float64
Highest 30 Min Rainfall (mm)     166111 non-null float64
Highest 60 Min Rainfall (mm)     166012 non-null float64
Highest 120 Min Rainfall (mm)    166015 non-null float64
Mean Temperature (°C)            72761 non-null float64
Maximum Temperature (°C)         78261 non-null float64
Minimum Temperature (°C)         78219 non-null float64
Mean Wind Speed (km/h)           78224 non-null float64
Max Wind Speed (km/h)            78656 non-null float64
Date                             228709 non-null object
Epiweek                          228709 non-null object
dtypes: float64(9), int64(3), object(3)
memory u

In [393]:
#Rename those long headers to shorter ones for ease of calling
df_weather.rename(columns={"Daily Rainfall Total (mm)": "total_rainfall", 
                            "Highest 30 Min Rainfall (mm)": "rainfall_30min",
                            "Highest 60 Min Rainfall (mm)": "rainfall_60min",
                            "Highest 120 Min Rainfall (mm)": "rainfall_120min",
                            "Mean Temperature (°C)": "temperature_mean",
                            "Maximum Temperature (°C)": "temperature_max",
                            "Minimum Temperature (°C)": "temperature_min",
                            "Mean Wind Speed (km/h)": "windspeed_mean",
                            "Max Wind Speed (km/h)": "windspeed_max"
                           }, inplace = True)

In [394]:
df_weather.info() #Before imputing

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228709 entries, 0 to 228708
Data columns (total 15 columns):
Station             228709 non-null object
Year                228709 non-null int64
Month               228709 non-null int64
Day                 228709 non-null int64
total_rainfall      220695 non-null float64
rainfall_30min      166111 non-null float64
rainfall_60min      166012 non-null float64
rainfall_120min     166015 non-null float64
temperature_mean    72761 non-null float64
temperature_max     78261 non-null float64
temperature_min     78219 non-null float64
windspeed_mean      78224 non-null float64
windspeed_max       78656 non-null float64
Date                228709 non-null object
Epiweek             228709 non-null object
dtypes: float64(9), int64(3), object(3)
memory usage: 26.2+ MB


In [395]:
columnlist = ["total_rainfall",  "rainfall_30min", "rainfall_60min", "rainfall_120min",
             "temperature_mean", "temperature_max", "temperature_min", 
              "windspeed_mean", "windspeed_max"]

In [396]:
for column in columnlist:
    df_weather[column] = df_weather.groupby(['Station','Year','Month'])[column].transform(lambda x: x.fillna(x.mean()))

In [397]:
df_weather.info() #After imputing

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228709 entries, 0 to 228708
Data columns (total 15 columns):
Station             228709 non-null object
Year                228709 non-null int64
Month               228709 non-null int64
Day                 228709 non-null int64
total_rainfall      226131 non-null float64
rainfall_30min      171833 non-null float64
rainfall_60min      171833 non-null float64
rainfall_120min     171833 non-null float64
temperature_mean    75358 non-null float64
temperature_max     80777 non-null float64
temperature_min     80777 non-null float64
windspeed_mean      80378 non-null float64
windspeed_max       81322 non-null float64
Date                228709 non-null object
Epiweek             228709 non-null object
dtypes: float64(9), int64(3), object(3)
memory usage: 26.2+ MB


As there are many other null values to be imputed, we will do a second round of imputation by using the mean of the similar months from other years.

In [398]:
for column in columnlist:
    df_weather[column] = df_weather.groupby(['Station','Month'])[column].transform(lambda x: x.fillna(x.mean()))

In [399]:
df_weather.info() #After second round of imputing

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228709 entries, 0 to 228708
Data columns (total 15 columns):
Station             228709 non-null object
Year                228709 non-null int64
Month               228709 non-null int64
Day                 228709 non-null int64
total_rainfall      228709 non-null float64
rainfall_30min      228525 non-null float64
rainfall_60min      228525 non-null float64
rainfall_120min     228525 non-null float64
temperature_mean    84240 non-null float64
temperature_max     84240 non-null float64
temperature_min     84240 non-null float64
windspeed_mean      84240 non-null float64
windspeed_max       84240 non-null float64
Date                228709 non-null object
Epiweek             228709 non-null object
dtypes: float64(9), int64(3), object(3)
memory usage: 26.2+ MB


Rainfall seems to be almost 100% imputed, temperature and windspeed are still less than 50% non-NA. For the rest, we will fill with the global mean.

In [400]:
df_weather[columnlist] = df_weather[columnlist].fillna(df_weather[columnlist].mean())

In [401]:
df_weather.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228709 entries, 0 to 228708
Data columns (total 15 columns):
Station             228709 non-null object
Year                228709 non-null int64
Month               228709 non-null int64
Day                 228709 non-null int64
total_rainfall      228709 non-null float64
rainfall_30min      228709 non-null float64
rainfall_60min      228709 non-null float64
rainfall_120min     228709 non-null float64
temperature_mean    228709 non-null float64
temperature_max     228709 non-null float64
temperature_min     228709 non-null float64
windspeed_mean      228709 non-null float64
windspeed_max       228709 non-null float64
Date                228709 non-null object
Epiweek             228709 non-null object
dtypes: float64(9), int64(3), object(3)
memory usage: 26.2+ MB


Finally, we have all the missing values imputed.

#### Import the NEA dengue cases data

This data reflects cases within clusters (defined as at least 2 reported dengue cases) in each geographical region in Singapore. The time period of the data scraped is mid-2013 to 2020. There are 39 unique regions in this dataframe.

In [402]:
df_cases_bylocations = pd.read_csv(filepath + 'final_location_based_data.csv')

In [403]:
df_cases_bylocations.head()

,epi_week_year,start,end,actual,ang mo kio,bedok,bishan,boon lay,bukit batok,bukit merah,...,sentosa,serangoon,sungei kadut,tampines,tanglin,tengah,toa payoh,tuas,woodlands,yishun
0,2013-W21,19/5/2013,25/5/2013,23/5/2013,0.0,63.0,5.0,4.0,3.0,0.0,...,0.0,10.0,0.0,203.0,0.0,0.0,15.0,0.0,3.0,81.0
1,2013-W22,26/5/2013,1/6/2013,27/5/2013,0.0,63.0,8.0,4.0,3.0,2.0,...,0.0,9.0,0.0,193.0,2.0,0.0,16.0,0.0,4.0,81.0
2,2013-W23,2/6/2013,8/6/2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-W24,9/6/2013,15/6/2013,11/6/2013,23.0,30.0,0.0,4.0,6.0,0.0,...,0.0,8.0,0.0,297.0,1.0,0.0,9.0,0.0,4.0,126.0
4,2013-W25,16/6/2013,22/6/2013,17/6/2013,11.0,28.0,7.0,4.0,4.0,8.0,...,0.0,17.0,0.0,325.0,5.0,0.0,7.0,0.0,0.0,145.0


The next step changes the column headers for the geographical regionss to rows - one row for each town


In [404]:
list_ = []
for region in list_regions:
    df_temp = pd.melt(df_cases_bylocations, 
                          id_vars =['epi_week_year', 'start', 'end', 'actual'], 
                          value_vars = region)
    list_.append(df_temp)

df_total_dengue_cases = pd.concat(list_, ignore_index = True)


In [405]:
df_total_dengue_cases.info() #There are null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19422 entries, 0 to 19421
Data columns (total 6 columns):
epi_week_year    15210 non-null object
start            15210 non-null object
end              15210 non-null object
actual           12285 non-null object
variable         19422 non-null object
value            12441 non-null float64
dtypes: float64(1), object(5)
memory usage: 910.5+ KB


In [406]:
df_total_dengue_cases.columns = ['epi_week_year','start', 'end', 'actual', 'town', 'dengue_cases']

In [407]:
df_total_dengue_cases.head()

,epi_week_year,start,end,actual,town,dengue_cases
0,2013-W21,19/5/2013,25/5/2013,23/5/2013,ang mo kio,0.0
1,2013-W22,26/5/2013,1/6/2013,27/5/2013,ang mo kio,0.0
2,2013-W23,2/6/2013,8/6/2013,NaN,ang mo kio,NaN
3,2013-W24,9/6/2013,15/6/2013,11/6/2013,ang mo kio,23.0
4,2013-W25,16/6/2013,22/6/2013,17/6/2013,ang mo kio,11.0


In [408]:
df_total_dengue_cases.dtypes

epi_week_year     object
start             object
end               object
actual            object
town              object
dengue_cases     float64
dtype: object

##### Dealing with null values
Impute all NaN values with backfill method.

In [409]:
df_total_dengue_cases = df_total_dengue_cases.bfill(axis ='rows')

In [410]:
df_total_dengue_cases.shape

(19422, 6)

In [411]:
df_total_dengue_cases.info() #The null values are mostly fixed with the back fill method.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19422 entries, 0 to 19421
Data columns (total 6 columns):
epi_week_year    19314 non-null object
start            19314 non-null object
end              19314 non-null object
actual           19314 non-null object
town             19422 non-null object
dengue_cases     19314 non-null float64
dtypes: float64(1), object(5)
memory usage: 910.5+ KB


Observing number of null values - these 108 values should be removed as they are meaningless to keep; only the 'town' value is filled up and the rest are NA values

In [412]:
#drop the null values
df_total_dengue_cases = df_total_dengue_cases.dropna()
df_total_dengue_cases.shape


(19314, 6)

In [413]:
df_total_dengue_cases.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19314 entries, 0 to 19313
Data columns (total 6 columns):
epi_week_year    19314 non-null object
start            19314 non-null object
end              19314 non-null object
actual           19314 non-null object
town             19314 non-null object
dengue_cases     19314 non-null float64
dtypes: float64(1), object(5)
memory usage: 1.0+ MB


#### Checkpoint

So far, we have 2 main dataframes, the `Weather` dataframe and `Total Dengue cases` dataframe. Since the `Station` column in the `Weather` dataframe and the `town` in the `Total Dengue cases` dataframe contain different list of values, we need to match the geographic regions in order to merge into one dataframe.

There are 2 ways to proceed:


1) One way is to split the whole of Singapore into broad regions i.e. North, South, East, West, Central.
- Every `town` in `Total Dengue cases` and every `Station` in `Weather` is matched to a broad region (North/South/west etc)
- Then, we will aggregate the values for each broad region to create the final dataset
- We can fit one model on all the regions represented (whole of SG), or run sub-models for each region

2) Another way (*preferred***) is to look through the list of Weather Stations and Towns, and create a lookup list to be used for merging the dataframes. This allows us to keep more granular data to have more rows of data for the eventual model. 
- As there may be a case where weather stations are in regions where no dengue cluster was reported, or vice versa (dengue cases are reported in places where there was no weather station), we will take only the intersection of the 2 lists of locations. 
- Also, there could be cases of many-to-one matching between the 2 lists of locations, for e.g. Tuas, Tuas South and Tuas West stations are all part of 'Tuas' region in the list of towns for theTotal Dengue Cases dataframe. We will deal with this in later steps

In [414]:
list_regions= list(df_cases_bylocations.columns) 
#Can also use the df_total_dengue_cases and find unique entries in the column for town
del list_regions[0:4]
print(list_regions)
print('\n Length:  ' + str(len(list_regions)))

['ang mo kio', 'bedok', 'bishan', 'boon lay', 'bukit batok', 'bukit merah', 'bukit panjang', 'bukit timah', 'changi', 'choa chu kang', 'city', 'clementi', 'geylang', 'hougang', 'jurong east', 'jurong island', 'jurong west', 'kallang', 'lim chu kang', 'mandai', 'marine parade', 'novena', 'pasir ris', 'paya lebar', 'punggol', 'queenstown', 'seletar', 'sembawang', 'sengkang', 'sentosa', 'serangoon', 'sungei kadut', 'tampines', 'tanglin', 'tengah', 'toa payoh', 'tuas', 'woodlands', 'yishun']

 Length:  39


In [415]:
list_weatherstations = list(set(df_weather['Station']))
print(list_weatherstations)
print('\n Length:  ' + str(len(list_weatherstations)))

['Pasir Ris (Central)', 'Buangkok', 'Mandai', 'Queenstown', 'Choa Chu Kang (West)', 'Choa Chu Kang (Central)', 'Chai Chee', 'Bukit Timah', 'Tuas West', 'Marine Parade', 'Admiralty', 'Somerset (Road)', 'Jurong Pier', 'Whampoa', 'Marina Barrage', 'Pasir Panjang', 'Khatib', 'Buona Vista', 'Tanjong Katong', 'Tuas South', 'Jurong Island', 'Pasir Ris (West)', 'Boon Lay (East)', 'Toa Payoh', 'Newton', 'Upper Peirce Reservoir', 'Semakau Island', 'Sentosa Island', 'Choa Chu Kang (South)', 'Tengah', 'Lower Peirce Reservoir', 'Lim Chu Kang', 'Nicoll Highway', 'Botanic Garden', 'Sembawang', 'Admiralty West', 'Clementi', 'Simei', 'East Coast Parkway', 'Yishun', 'Kent Ridge', 'Tuas', 'Tai Seng', 'Ang Mo Kio', 'Jurong (West)', 'Kranji Reservoir', 'Bukit Panjang', 'Jurong (East)', 'Kampong Bahru', 'Macritchie Reservoir', 'Paya Lebar', 'Serangoon North', 'Seletar', 'Tanjong Pagar', 'Punggol', 'Serangoon', 'Dhoby Ghaut', 'Upper Thomson', 'Changi', 'Pulau Ubin', 'Boon Lay (West)', 'Jurong (North)', 'Ulu 

In [416]:
df_total_dengue_cases.groupby('town').mean() 

,dengue_cases
town,
ang mo kio,45.991968
bedok,69.000000
bishan,22.626506
boon lay,1.718876
bukit batok,9.636546
bukit merah,3.865462
bukit panjang,21.558233
bukit timah,10.018072
changi,12.000000


Note to self:EDA this to show the town with largest number of dengue incidences over the year. Also can split by years

In [417]:
df_weather.groupby('Station').mean() 

,Year,Month,Day,total_rainfall,rainfall_30min,rainfall_60min,rainfall_120min,temperature_mean,temperature_max,temperature_min,windspeed_mean,windspeed_max
Station,,,,,,,,,,,,
Admiralty,2016.982815,6.460955,15.730099,6.610667,3.932260,4.869191,5.507810,27.680922,31.860918,24.989589,9.421773,35.762590
Admiralty West,2015.332928,6.353609,15.732360,6.697923,3.810777,4.728076,5.323158,27.953434,31.714301,25.284095,8.436315,32.783820
Ang Mo Kio,2016.960371,6.480060,15.728367,6.975168,4.164612,5.152199,5.809086,27.869856,31.909446,25.133092,6.921642,31.434993
Boon Lay (East),2013.550938,6.408177,15.735255,6.793385,3.813839,4.765226,5.438994,28.233449,31.953027,25.542549,6.991903,32.412069
Boon Lay (West),2014.637599,6.438650,15.724803,7.262728,4.197699,5.263113,6.105475,27.953434,31.714301,25.284095,8.436315,32.783820
Botanic Garden,2016.960371,6.480060,15.728367,7.076398,4.123492,5.158368,5.905978,27.953434,31.714301,25.284095,8.436315,32.783820
Buangkok,2015.178558,6.342174,15.721703,6.314296,3.664136,4.464524,4.967027,27.953434,31.714301,25.284095,8.436315,32.783820
Bukit Panjang,2016.915500,6.509544,15.738865,7.515390,4.470101,5.569496,6.316267,27.953434,31.714301,25.284095,8.436315,32.783820
Bukit Timah,2017.038561,6.399845,15.723602,6.941601,4.094899,5.059674,5.774977,27.953434,31.714301,25.284095,8.436315,32.783820


We created a lookup table showing the regions that each town belongs to, and how it maps to each weather station. 
As some towns/regions were represented in one dataset but not the other, we only take the intersection.
For some towns e.g. Choa Chu Kang, there can be a one-to-many relationship to the weather station. We need to be mindful of this and take the mean values for such towns during data cleaning and merging the dataframes. 

An easy way would be to drop such towns. However, the more comprehensive way would be to for these 'many-to-one' weather station to town matches, we do some work to take the aggregate mean of the multiple weather stations mapped to the 1 town, before merging both dataframes on the town and the epiweek. This way, there is greater data retention.

In [418]:
lookuptable = pd.read_excel(filepath + 'Lookup_weather_stations.xlsx')

In [419]:
lookuptable

,Town,Region,Weather Station
0,ang mo kio,Central,Ang Mo Kio
1,boon lay,West,Boon Lay (West)
2,boon lay,West,Boon Lay (East)
3,bukit panjang,West,Bukit Panjang
4,bukit timah,South,Bukit Timah
5,changi,East,Changi
6,choa chu kang,West,Choa Chu Kang (Central)
7,choa chu kang,West,Choa Chu Kang (South)
8,choa chu kang,West,Choa Chu Kang (West)
9,clementi,West,Clementi


In [420]:
list_regions_sub = set(list(lookuptable['Town']))
list_weatherstations_sub = list(lookuptable['Weather Station'])

Do some merging by location and dates to achieve the final dataframe for modelling

In [421]:
df_weather['Station Indicator'] = np.where(df_weather['Station'].isin(list_weatherstations_sub), 1, np.nan)
df_weather = df_weather[df_weather['Station Indicator'] == 1] #Filter for those in list
df_weather.head()

,Station,Year,Month,Day,total_rainfall,rainfall_30min,rainfall_60min,rainfall_120min,temperature_mean,temperature_max,temperature_min,windspeed_mean,windspeed_max,Date,Epiweek,Station Indicator
0,Punggol,2012,1,1,9.8,2.599234,3.389148,4.111853,27.953434,31.714301,25.284095,8.436315,32.78382,1/1/2012,2012-W01,1.0
1,Punggol,2012,1,2,0.0,2.599234,3.389148,4.111853,27.953434,31.714301,25.284095,8.436315,32.78382,2/1/2012,2012-W01,1.0
2,Punggol,2012,1,3,0.0,2.599234,3.389148,4.111853,27.953434,31.714301,25.284095,8.436315,32.78382,3/1/2012,2012-W01,1.0
3,Punggol,2012,1,4,0.0,2.599234,3.389148,4.111853,27.953434,31.714301,25.284095,8.436315,32.78382,4/1/2012,2012-W01,1.0
4,Punggol,2012,1,5,0.0,2.599234,3.389148,4.111853,27.953434,31.714301,25.284095,8.436315,32.78382,5/1/2012,2012-W01,1.0


In [422]:
df_weather_combined = df_weather.merge(lookuptable, left_on = 'Station', right_on = 'Weather Station')
df_weather_combined.drop(columns = ['Weather Station', 'Station Indicator'], inplace= True)

In [423]:
df_weather_combined.head()

,Station,Year,Month,Day,total_rainfall,rainfall_30min,rainfall_60min,rainfall_120min,temperature_mean,temperature_max,temperature_min,windspeed_mean,windspeed_max,Date,Epiweek,Town,Region
0,Punggol,2012,1,1,9.8,2.599234,3.389148,4.111853,27.953434,31.714301,25.284095,8.436315,32.78382,1/1/2012,2012-W01,punggol,North East
1,Punggol,2012,1,2,0.0,2.599234,3.389148,4.111853,27.953434,31.714301,25.284095,8.436315,32.78382,2/1/2012,2012-W01,punggol,North East
2,Punggol,2012,1,3,0.0,2.599234,3.389148,4.111853,27.953434,31.714301,25.284095,8.436315,32.78382,3/1/2012,2012-W01,punggol,North East
3,Punggol,2012,1,4,0.0,2.599234,3.389148,4.111853,27.953434,31.714301,25.284095,8.436315,32.78382,4/1/2012,2012-W01,punggol,North East
4,Punggol,2012,1,5,0.0,2.599234,3.389148,4.111853,27.953434,31.714301,25.284095,8.436315,32.78382,5/1/2012,2012-W01,punggol,North East


In [424]:
df_weather_combined.shape #This will decrease by at least (divide by 7) = ~15K rows

(115830, 17)

In [425]:
list_manyto1 = ['boon lay', 'choa chu kang', 'pasir ris', 'serangoon', 'tuas']

Intuition would be to create new columns using np.where to apply the mean value of all those from the same town, epiweek. Then, drop the unnecessary columns of Station, Year, Month, Day (Keep the Date column for merging with gtrends later on). This applies to the whole dataset as there are 7 days in a week, even for those that are not in the many-to-one list.


Then, remove the duplicate rows that have the same Epiweek and Town, keeping just the first column. Remember to check the shape of the dataframe at all times.

In [426]:
columnlist

['total_rainfall',
 'rainfall_30min',
 'rainfall_60min',
 'rainfall_120min',
 'temperature_mean',
 'temperature_max',
 'temperature_min',
 'windspeed_mean',
 'windspeed_max']

In [427]:
df_weather_combined['total_rainfall'] = df_weather_combined.groupby(['Epiweek', 'Town'])['total_rainfall'].transform(lambda x: x.sum())


In [428]:
df_weather_combined['rainfall_30min'] = df_weather_combined.groupby(['Epiweek', 'Town'])['rainfall_30min'].transform(lambda x: x.max())
df_weather_combined['rainfall_60min'] = df_weather_combined.groupby(['Epiweek', 'Town'])['rainfall_60min'].transform(lambda x: x.max())
df_weather_combined['rainfall_120min'] = df_weather_combined.groupby(['Epiweek', 'Town'])['rainfall_120min'].transform(lambda x: x.max())


In [429]:
df_weather_combined['temperature_mean'] = df_weather_combined.groupby(['Epiweek', 'Town'])['temperature_mean'].transform(lambda x: x.mean())
df_weather_combined['temperature_max'] = df_weather_combined.groupby(['Epiweek', 'Town'])['temperature_max'].transform(lambda x: x.max())
df_weather_combined['temperature_min'] = df_weather_combined.groupby(['Epiweek', 'Town'])['temperature_min'].transform(lambda x: x.min())

df_weather_combined['windspeed_mean'] = df_weather_combined.groupby(['Epiweek', 'Town'])['windspeed_mean'].transform(lambda x: x.mean())
df_weather_combined['windspeed_max'] = df_weather_combined.groupby(['Epiweek', 'Town'])['windspeed_max'].transform(lambda x: x.max())

In [430]:
df_weather_combined.head()

,Station,Year,Month,Day,total_rainfall,rainfall_30min,rainfall_60min,rainfall_120min,temperature_mean,temperature_max,temperature_min,windspeed_mean,windspeed_max,Date,Epiweek,Town,Region
0,Punggol,2012,1,1,13.8,2.599234,3.389148,4.111853,27.953434,31.714301,25.284095,8.436315,32.78382,1/1/2012,2012-W01,punggol,North East
1,Punggol,2012,1,2,13.8,2.599234,3.389148,4.111853,27.953434,31.714301,25.284095,8.436315,32.78382,2/1/2012,2012-W01,punggol,North East
2,Punggol,2012,1,3,13.8,2.599234,3.389148,4.111853,27.953434,31.714301,25.284095,8.436315,32.78382,3/1/2012,2012-W01,punggol,North East
3,Punggol,2012,1,4,13.8,2.599234,3.389148,4.111853,27.953434,31.714301,25.284095,8.436315,32.78382,4/1/2012,2012-W01,punggol,North East
4,Punggol,2012,1,5,13.8,2.599234,3.389148,4.111853,27.953434,31.714301,25.284095,8.436315,32.78382,5/1/2012,2012-W01,punggol,North East


In [431]:
df_weather_combined.shape

(115830, 17)

In [432]:
df_weather_combined.groupby(['Epiweek', 'Town']).count()

Station  Year  Month  Day  total_rainfall  \
Epiweek  Town                                                       
2012-W01 ang mo kio           7     7      7    7               7   
         boon lay            14    14     14   14              14   
         bukit panjang        7     7      7    7               7   
         bukit timah          7     7      7    7               7   
         changi               7     7      7    7               7   
         choa chu kang       21    21     21   21              21   
         clementi             7     7      7    7               7   
         jurong east          7     7      7    7               7   
         jurong island        7     7      7    7               7   
         jurong west          7     7      7    7               7   
         lim chu kang         7     7      7    7               7   
         marine parade        7     7      7    7               7   
         pasir ris           14    14     14   14              14   
         paya lebar           7     7      7    7               7   
         punggol              7     7      7    7               7   
         queenstown           7     7      7    7               7   
         seletar              7     7      7    7               7   
         sembawang            7     7      7    7               7   
         sentosa              7     7      7    7               7   
         serangoon           14    14     14   14              14   
         tengah               7     7      7    7               7   
         toa payoh            7     7      7    7               7   
         tuas                21    21     21   21              21   
         yishun               7     7      7    7               7   
2012-W02 ang mo kio           7     7      7    7               7   
         boon lay            14    14     14   14              14   
         bukit panjang        7     7      7    7               7   
         bukit timah          7     7      7    7               7   
         changi               7     7      7    7               7   
         choa chu kang       21    21     21   21              21   
...                         ...   ...    ...  ...             ...   
2022-W47 queenstown           7     7      7    7               7   
         seletar              7     7      7    7               7   
         sembawang            7     7      7    7               7   
         sentosa              7     7      7    7               7   
         serangoon            7     7      7    7               7   
         tengah               7     7      7    7               7   
         toa payoh            7     7      7    7               7   
         tuas                21    21     21   21              21   
2022-W48 ang mo kio           4     4      4    4               4   
         bukit panjang        4     4      4    4               4   
         bukit timah          4     4      4    4               4   
         changi               4     4      4    4               4   
         choa chu kang        8     8      8    8               8   
         clementi             4     4      4    4               4   
         jurong island        4     4      4    4               4   
         jurong west          4     4      4    4               4   
         lim chu kang         4     4      4    4               4   
         mandai               4     4      4    4               4   
         marine parade        4     4      4    4               4   
         pasir ris            8     8      8    8               8   
         paya lebar           4     4      4    4               4   
         punggol              4     4      4    4               4   
         queenstown           4     4      4    4               4   
         seletar              4     4      4    4               4   
         sembawang            4     4      4    4               4   
         sentosa      

In [433]:
df_weather_combined_unique = df_weather_combined.drop_duplicates(subset=['Epiweek', 'Town'], keep='first')

In [434]:
df_weather_combined_unique.shape

(13360, 17)

In [435]:
df_weather_combined_unique.head()

,Station,Year,Month,Day,total_rainfall,rainfall_30min,rainfall_60min,rainfall_120min,temperature_mean,temperature_max,temperature_min,windspeed_mean,windspeed_max,Date,Epiweek,Town,Region
0,Punggol,2012,1,1,13.8,2.599234,3.389148,4.111853,27.953434,31.714301,25.284095,8.436315,32.78382,1/1/2012,2012-W01,punggol,North East
7,Punggol,2012,1,8,29.6,2.599234,3.389148,4.111853,27.953434,31.714301,25.284095,8.436315,32.78382,8/1/2012,2012-W02,punggol,North East
14,Punggol,2012,1,15,66.4,2.599234,3.389148,4.111853,27.953434,31.714301,25.284095,8.436315,32.78382,15/1/2012,2012-W03,punggol,North East
21,Punggol,2012,1,22,16.0,2.599234,3.389148,4.111853,27.953434,31.714301,25.284095,8.436315,32.78382,22/1/2012,2012-W04,punggol,North East
28,Punggol,2012,1,29,53.2,2.599234,3.389148,4.111853,27.953434,31.714301,25.284095,8.436315,32.78382,29/1/2012,2012-W05,punggol,North East


In [436]:
#Shortcut to drop those many-to-one
#df_weather_combined2 = df_weather_combined
#df_weather_combined2['Indicator'] = np.where(df_weather_combined2['Town'].isin(list_manyto1), np.nan, 1)
#df_weather_combined2 = df_weather_combined2[df_weather_combined2['Indicator'] == 1] #Filter for those in list
#df_weather_combined2.drop(columns = ['Indicator'], inplace = True)

In [437]:
grouped = df_weather_combined_unique.groupby(['Town', 'Epiweek'])
grouped.max()


Station  Year  Month  Day  total_rainfall  \
Town       Epiweek                                                  
ang mo kio 2012-W01  Ang Mo Kio  2012      1    1        3.800000   
           2012-W02  Ang Mo Kio  2012      1    8       32.400000   
           2012-W03  Ang Mo Kio  2012      1   15      101.400000   
           2012-W04  Ang Mo Kio  2012      1   22       30.400000   
           2012-W05  Ang Mo Kio  2012      1   29       85.600000   
           2012-W06  Ang Mo Kio  2012      2    5        0.000000   
           2012-W07  Ang Mo Kio  2012      2   12       25.800000   
           2012-W08  Ang Mo Kio  2012      2   19       21.200000   
           2012-W09  Ang Mo Kio  2012      3    1       28.600000   
           2012-W10  Ang Mo Kio  2012      3    4       24.400000   
           2012-W11  Ang Mo Kio  2012      3   11       69.600000   
           2012-W12  Ang Mo Kio  2012      3   18       38.400000   
           2012-W13  Ang Mo Kio  2012      3   25      104.400000   
           2012-W14  Ang Mo Kio  2012      4    1       87.000000   
           2012-W15  Ang Mo Kio  2012      4    8      117.800000   
           2012-W16  Ang Mo Kio  2012      4   15       37.600000   
           2012-W17  Ang Mo Kio  2012      4   22        2.000000   
           2012-W18  Ang Mo Kio  2012      5    1       87.600000   
           2012-W19  Ang Mo Kio  2012      5    6       81.200000   
           2012-W20  Ang Mo Kio  2012      5   13       21.800000   
           2012-W21  Ang Mo Kio  2012      5   20       44.200000   
           2012-W22  Ang Mo Kio  2012      6    1       25.000000   
           2012-W23  Ang Mo Kio  2012      6    3       50.000000   
           2012-W24  Ang Mo Kio  2012      6   10        0.000000   
           2012-W25  Ang Mo Kio  2012      6   17       16.000000   
           2012-W26  Ang Mo Kio  2012      6   24       78.800000   
           2012-W27  Ang Mo Kio  2012      7    1      115.600000   
           2012-W28  Ang Mo Kio  2012      7    8       27.400000   
           2012-W29  Ang Mo Kio  2012      7   15       30.600000   
           2012-W30  Ang Mo Kio  2012      7   22        0.400000   
...                         ...   ...    ...  ...             ...   
yishun     2020-W41      Yishun  2020     10    4       15.000000   
           2020-W42      Yishun  2020     10   11       25.800000   
           2020-W43      Yishun  2020     10   18       22.800000   
           2020-W44      Yishun  2020     10   25       47.600000   
           2020-W45      Yishun  2020     11    1       90.200000   
           2020-W46      Yishun  2020     11    8       31.400000   
           2020-W47      Yishun  2020     11   15      108.600000   
           2020-W48      Yishun  2020     11   22       41.400000   
           2020-W49      Yishun  2020     12    1       38.600000   
           2020-W50      Yishun  2020     12    6       73.400000   
           2020-W51      Yishun  2020     12   13       52.400000   
           2020-W52      Yishun  2020     12   20       57.200000   
           2020-W53      Yishun  2020     12   27      256.200000   
           2021-W01      Yishun  2021      1    3       44.800000   
           2021-W02      Yishun  2021      1   10      156.737931   
           2021-W03      Yishun  2021      1   17       72.600000   
           2021-W04      Yishun  2021      1   24        2.800000   
           2021-W05      Yishun  2021      2    1        0.000000   
           2021-W06      Yishun  2021      2    7       27.000000   
           2021-W07      Yishun  2021      2   14        0.400000   
           2021-W08      Yishun  2021      2   21        6.400000   
           2021-W09      Yishun  2021      3    1        0.000000   
           2021-W10      Yishun  2021      3    7       92.800000   
           2021-W11      Yishun  2021      3   14       56.533333   
           2021-W12      Yishun  2021      3   21       52.266667   
           2021-W13   

Inner Join with `df_total_dengue_cases`

In [438]:
df_total_dengue_cases.head()

,epi_week_year,start,end,actual,town,dengue_cases
0,2013-W21,19/5/2013,25/5/2013,23/5/2013,ang mo kio,0.0
1,2013-W22,26/5/2013,1/6/2013,27/5/2013,ang mo kio,0.0
2,2013-W23,2/6/2013,8/6/2013,11/6/2013,ang mo kio,23.0
3,2013-W24,9/6/2013,15/6/2013,11/6/2013,ang mo kio,23.0
4,2013-W25,16/6/2013,22/6/2013,17/6/2013,ang mo kio,11.0


In [439]:
df_weather_combined_unique.head()

,Station,Year,Month,Day,total_rainfall,rainfall_30min,rainfall_60min,rainfall_120min,temperature_mean,temperature_max,temperature_min,windspeed_mean,windspeed_max,Date,Epiweek,Town,Region
0,Punggol,2012,1,1,13.8,2.599234,3.389148,4.111853,27.953434,31.714301,25.284095,8.436315,32.78382,1/1/2012,2012-W01,punggol,North East
7,Punggol,2012,1,8,29.6,2.599234,3.389148,4.111853,27.953434,31.714301,25.284095,8.436315,32.78382,8/1/2012,2012-W02,punggol,North East
14,Punggol,2012,1,15,66.4,2.599234,3.389148,4.111853,27.953434,31.714301,25.284095,8.436315,32.78382,15/1/2012,2012-W03,punggol,North East
21,Punggol,2012,1,22,16.0,2.599234,3.389148,4.111853,27.953434,31.714301,25.284095,8.436315,32.78382,22/1/2012,2012-W04,punggol,North East
28,Punggol,2012,1,29,53.2,2.599234,3.389148,4.111853,27.953434,31.714301,25.284095,8.436315,32.78382,29/1/2012,2012-W05,punggol,North East


In [440]:
df = df_total_dengue_cases.merge(df_weather_combined_unique, left_on = ['town', 'epi_week_year'], right_on = ['Town', 'Epiweek'], how = 'inner')

In [441]:
#check shape before and after merging
print(df_total_dengue_cases.shape)
print(df_weather_combined_unique.shape)
print(df.shape)


(19314, 6)
(13360, 17)
(11811, 23)


No idea why there is massive number of duplicates (109 rows for only the epiweek 2013-W21 for each of the 25 towns, so I shall drop duplicates again. This should drop around 2600 rows.



In [442]:
df2 = df.drop_duplicates(subset=['Epiweek', 'Town'], keep='first')

In [443]:
df2.head() 

,epi_week_year,start,end,actual,town,dengue_cases,Station,Year,Month,Day,...,rainfall_120min,temperature_mean,temperature_max,temperature_min,windspeed_mean,windspeed_max,Date,Epiweek,Town,Region
0,2013-W21,19/5/2013,25/5/2013,23/5/2013,ang mo kio,0.0,Ang Mo Kio,2013,5,19,...,5.854992,28.300000,34.9,22.7,4.100000,38.9,19/5/2013,2013-W21,ang mo kio,Central
109,2013-W22,26/5/2013,1/6/2013,27/5/2013,ang mo kio,0.0,Ang Mo Kio,2013,5,26,...,5.854992,27.685714,34.3,23.0,4.371429,64.8,26/5/2013,2013-W22,ang mo kio,Central
110,2013-W23,2/6/2013,8/6/2013,11/6/2013,ang mo kio,23.0,Ang Mo Kio,2013,6,2,...,5.233173,27.985714,35.5,23.2,4.357143,48.2,2/6/2013,2013-W23,ang mo kio,Central
111,2013-W24,9/6/2013,15/6/2013,11/6/2013,ang mo kio,23.0,Ang Mo Kio,2013,6,9,...,5.233173,29.742857,34.1,24.2,6.328571,39.2,9/6/2013,2013-W24,ang mo kio,Central
112,2013-W25,16/6/2013,22/6/2013,17/6/2013,ang mo kio,11.0,Ang Mo Kio,2013,6,16,...,5.233173,30.385714,34.6,26.8,6.300000,43.9,16/6/2013,2013-W25,ang mo kio,Central


In [444]:
df2.shape

(9219, 23)

In [445]:
df2.isna().sum() #Eliminated or imputed all null values

epi_week_year       0
start               0
end                 0
actual              0
town                0
dengue_cases        0
Station             0
Year                0
Month               0
Day                 0
total_rainfall      0
rainfall_30min      0
rainfall_60min      0
rainfall_120min     0
temperature_mean    0
temperature_max     0
temperature_min     0
windspeed_mean      0
windspeed_max       0
Date                0
Epiweek             0
Town                0
Region              0
dtype: int64

In [446]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9219 entries, 0 to 11810
Data columns (total 23 columns):
epi_week_year       9219 non-null object
start               9219 non-null object
end                 9219 non-null object
actual              9219 non-null object
town                9219 non-null object
dengue_cases        9219 non-null float64
Station             9219 non-null object
Year                9219 non-null int64
Month               9219 non-null int64
Day                 9219 non-null int64
total_rainfall      9219 non-null float64
rainfall_30min      9219 non-null float64
rainfall_60min      9219 non-null float64
rainfall_120min     9219 non-null float64
temperature_mean    9219 non-null float64
temperature_max     9219 non-null float64
temperature_min     9219 non-null float64
windspeed_mean      9219 non-null float64
windspeed_max       9219 non-null float64
Date                9219 non-null object
Epiweek             9219 non-null object
Town                9219

In [447]:
df2.drop(columns = ['town', 'end', 'actual'],inplace = True)

C:\Users\gohch\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [448]:
df2.to_excel('Output after Merged.xlsx', index=False)

#### Import the rescaled Google Trends data 
Google Trends data were scraped for the keywords 'Dengue fever' and 'Wolbachia'. This was done for weekly data and there were no missing values.

In [449]:
# Load the data from the JSON file
with open('data/rescaled_dengue fever.json', 'r') as f:
    data = json.load(f)

# Initialize an empty list to store the reformatted data
formatted_data = []

# Loop through each year and the data for that year
for year, year_data in data:
    # Convert the year data into a Pandas DataFrame
    dengue_fever = pd.DataFrame.from_dict(year_data, orient='index')
    dengue_fever.index.name = 'date'
    dengue_fever.reset_index(inplace=True)
    dengue_fever['date'] = pd.to_datetime(dengue_fever['date'])
    dengue_fever['year'] = year
    formatted_data.append(dengue_fever)

# Concatenate all of the year data into a single DataFrame
dengue_fever = pd.concat(formatted_data)


In [450]:
dengue_fever.columns = ['date', 'freq_dengue_gtrends', 'year']

In [451]:
dengue_fever.head()

,date,freq_dengue_gtrends,year
0,2012-01-01,84.658373,2012
1,2012-01-08,77.483935,2012
2,2012-01-15,28.697754,2012
3,2012-01-22,77.483935,2012
4,2012-01-29,50.221069,2012


Google Trends - Wolbachia

In [452]:
with open('data/rescaled_wolbachia.json', 'r') as f:
    data = json.load(f)

# Initialize an empty list to store the reformatted data
formatted_data = []

# Loop through each year and the data for that year
for year, year_data in data:
    # Convert the year data into a Pandas DataFrame
    wolbachia = pd.DataFrame.from_dict(year_data, orient='index')
    wolbachia.index.name = 'date'
    wolbachia.reset_index(inplace=True)
    wolbachia['date'] = pd.to_datetime(wolbachia['date'])
    wolbachia['year'] = year
    formatted_data.append(wolbachia)

# Concatenate all of the year data into a single DataFrame
wolbachia = pd.concat(formatted_data)


In [453]:
wolbachia.columns = ['date', 'freq_wolbachia_gtrends', 'year']

In [454]:
wolbachia.head()

,date,freq_wolbachia_gtrends,year
0,2012-01-01,0.0,2012
1,2012-01-08,0.0,2012
2,2012-01-15,0.0,2012
3,2012-01-22,0.0,2012
4,2012-01-29,0.0,2012


In [455]:
dengue_fever.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 488 entries, 0 to 54
Data columns (total 3 columns):
date                   488 non-null datetime64[ns]
freq_dengue_gtrends    488 non-null float64
year                   488 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 15.2 KB


In [456]:
wolbachia.info() #There are no null values for either of the Google trends data


<class 'pandas.core.frame.DataFrame'>
Int64Index: 488 entries, 0 to 54
Data columns (total 3 columns):
date                      488 non-null datetime64[ns]
freq_wolbachia_gtrends    488 non-null float64
year                      488 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 15.2 KB


Merging both into one dataframe

In [457]:
freq_gtrends = dengue_fever.merge(wolbachia, on =['date','year'])
freq_gtrends.head()

,date,freq_dengue_gtrends,year,freq_wolbachia_gtrends
0,2012-01-01,84.658373,2012,0.0
1,2012-01-08,77.483935,2012,0.0
2,2012-01-15,28.697754,2012,0.0
3,2012-01-22,77.483935,2012,0.0
4,2012-01-29,50.221069,2012,0.0


In [458]:
print(freq_gtrends.shape) # No loss or extra columns 

(488, 4)


In [459]:
freq_gtrends.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 488 entries, 0 to 487
Data columns (total 4 columns):
date                      488 non-null datetime64[ns]
freq_dengue_gtrends       488 non-null float64
year                      488 non-null int64
freq_wolbachia_gtrends    488 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 19.1 KB


In order to merge with the df2 dataframe, we will need to convert the date to type 'str', or add a column called Epiweek. We shall try to first directly convert to str and replace all '-' with '/' to match the format of df2 'Date' column.

In [460]:
test=freq_gtrends['date'][1]

In [461]:
test

Timestamp('2012-01-08 00:00:00')

In [462]:
#freq_gtrends['date_new'] = freq_gtrends['date'].apply(lambda x: str(x).replace('-','/'))
freq_gtrends['date_new'] = freq_gtrends['date'].apply(lambda x: x.strftime("%#d/%#m/%Y"))

In [463]:
freq_gtrends['date_new'].tail()

483     6/12/2020
484    13/12/2020
485    20/12/2020
486    27/12/2020
487      3/1/2021
Name: date_new, dtype: object

### Merging Google Trends into the final dataframe

In [464]:
df2.tail()

,epi_week_year,start,dengue_cases,Station,Year,Month,Day,total_rainfall,rainfall_30min,rainfall_60min,rainfall_120min,temperature_mean,temperature_max,temperature_min,windspeed_mean,windspeed_max,Date,Epiweek,Town,Region
11806,2020-W41,4/10/2020,0.0,Yishun,2020,10,4,15.0,2.6,3.4,5.0,27.953434,31.714301,25.284095,8.436315,32.78382,4/10/2020,2020-W41,yishun,North
11807,2020-W42,11/10/2020,0.0,Yishun,2020,10,11,25.8,14.0,14.0,14.0,27.953434,31.714301,25.284095,8.436315,32.78382,11/10/2020,2020-W42,yishun,North
11808,2020-W43,18/10/2020,0.0,Yishun,2020,10,18,22.8,9.4,9.4,9.4,27.953434,31.714301,25.284095,8.436315,32.78382,18/10/2020,2020-W43,yishun,North
11809,2020-W44,25/10/2020,0.0,Yishun,2020,10,25,47.6,26.2,26.6,26.8,27.953434,31.714301,25.284095,8.436315,32.78382,25/10/2020,2020-W44,yishun,North
11810,2020-W45,1/11/2020,0.0,Yishun,2020,11,1,90.2,29.4,34.4,37.0,27.953434,31.714301,25.284095,8.436315,32.78382,1/11/2020,2020-W45,yishun,North


In [465]:
df3 = df2.merge(freq_gtrends, left_on = 'Date', right_on = 'date_new', how ='left')

In [466]:
df3.isnull().sum()

epi_week_year               0
start                       0
dengue_cases                0
Station                     0
Year                        0
Month                       0
Day                         0
total_rainfall              0
rainfall_30min              0
rainfall_60min              0
rainfall_120min             0
temperature_mean            0
temperature_max             0
temperature_min             0
windspeed_mean              0
windspeed_max               0
Date                        0
Epiweek                     0
Town                        0
Region                      0
date                      918
freq_dengue_gtrends       918
year                      918
freq_wolbachia_gtrends    918
date_new                  918
dtype: int64

Finally, impute then null values for `freq_dengue_gtrends` and `freq_wolbachia_gtrends` with global means.
Drop the columns `Year`,`Month`,`Day`,`Station` (this is a repeat of Town column).

In [467]:
df3['freq_dengue_gtrends'] = df3['freq_dengue_gtrends'].fillna(df3['freq_dengue_gtrends'].mean())

In [468]:
df3['freq_wolbachia_gtrends'] = df3['freq_wolbachia_gtrends'].fillna(df3['freq_wolbachia_gtrends'].mean())

In [469]:
df3.isnull().sum()


epi_week_year               0
start                       0
dengue_cases                0
Station                     0
Year                        0
Month                       0
Day                         0
total_rainfall              0
rainfall_30min              0
rainfall_60min              0
rainfall_120min             0
temperature_mean            0
temperature_max             0
temperature_min             0
windspeed_mean              0
windspeed_max               0
Date                        0
Epiweek                     0
Town                        0
Region                      0
date                      918
freq_dengue_gtrends         0
year                      918
freq_wolbachia_gtrends      0
date_new                  918
dtype: int64

In [470]:
df3.drop(columns = ['Station','date', 'year','date_new'], inplace = True)

In [471]:
df3.isnull().sum()

epi_week_year             0
start                     0
dengue_cases              0
Year                      0
Month                     0
Day                       0
total_rainfall            0
rainfall_30min            0
rainfall_60min            0
rainfall_120min           0
temperature_mean          0
temperature_max           0
temperature_min           0
windspeed_mean            0
windspeed_max             0
Date                      0
Epiweek                   0
Town                      0
Region                    0
freq_dengue_gtrends       0
freq_wolbachia_gtrends    0
dtype: int64

In [472]:
df3.shape

(9494, 21)

In [473]:
df3.to_excel('Output after Merged.xlsx', index=False)